<h4> Objectives</h4>
<ul>
<li> Finding Similar Conversations
</ul>

In [15]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn
import os
from gensim.models.keyedvectors import KeyedVectors
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import pickle
import gensim
from gensim.models.doc2vec import TaggedDocument
import random
import numpy as np
import ast
from tqdm import tqdm
%matplotlib inline

In [3]:
lines = pd.read_json("dataset"+os.sep+"movie_lines.json")

In [4]:
lines.head(3)

,character_id,character_name,line_id,movie_id,text
0,u0,BIANCA,L1045,m0,They do not!
1,u2,CAMERON,L1044,m0,They do to!
10,u2,CAMERON,L868,m0,"The ""real you""."


In [14]:
lines_dict = {}
for idx,row in lines.iterrows():
    lines_dict[row["line_id"]]=row["text"]

In [5]:
conversations = pd.read_json("dataset"+os.sep+"movie_conversations.json")

In [6]:
conversations.head(3)

,character_id_1,character_id_2,line_ids,movie_id
0,u0,u2,"['L194', 'L195', 'L196', 'L197']",m0
1,u0,u2,"['L198', 'L199']",m0
10,u0,u2,"['L367', 'L368']",m0


In [8]:
conversations["dialogues_text"]=""

In [16]:
for idx,row in tqdm(conversations.iterrows()):
    dialogues = ast.literal_eval(row["line_ids"])
    dialogues_text = [lines_dict[d]
                      for d in dialogues]
    dialogues_text = " ".join(dialogues_text)
    conversations.set_value(idx,"dialogues_text",dialogues_text)

83097it [00:12, 6432.67it/s]


In [17]:
conversations.head(3)

,character_id_1,character_id_2,line_ids,movie_id,dialogues_text
0,u0,u2,"['L194', 'L195', 'L196', 'L197']",m0,Can we make this quick? Roxanne Korrine and A...
1,u0,u2,"['L198', 'L199']",m0,You're asking me out. That's so cute. What's ...
10,u0,u2,"['L367', 'L368']",m0,How do you get your hair to look like that? Eb...


In [21]:
def clean_text(corpus):
    punctuation = """.,?!:;(){}[]-"""
    corpus = [z.lower().replace("\n","").strip() for z in corpus.split(" ")]
    #corpus = [z for z in corpus if not z in stopwords.words('english')]
    for c in punctuation:
        corpus = [z.replace(c," ") for z in corpus]
    return corpus

In [22]:
conversations["dialogues_text"] = conversations["dialogues_text"].apply(lambda x:clean_text(x))

In [23]:
conversations.head(3)

,character_id_1,character_id_2,line_ids,movie_id,dialogues_text
0,u0,u2,"['L194', 'L195', 'L196', 'L197']",m0,"[can, we, make, this, quick , , roxanne, korri..."
1,u0,u2,"['L198', 'L199']",m0,"[you're, asking, me, out , , that's, so, cute ..."
10,u0,u2,"['L367', 'L368']",m0,"[how, do, you, get, your, hair, to, look, like..."


In [24]:
conversations.to_json("dataset"+os.sep+"conversations_agg.json")

In [61]:
conversations["new_index"]=""

In [62]:
for idx,row in conversations.iterrows():
    id_text = row["movie_id"]+"_"+row["character_id_1"]+"_"+row["character_id_2"]+"_"+str(idx)
    conversations.set_value(idx,"new_index",id_text)

In [37]:
x_train = [ TaggedDocument(words = row["dialogues_text"],tags = [row["movie_id"]+"_"+row["character_id_1"]+"_"+row["character_id_2"]+"_"+str(idx)])
            for idx,row in conversations.iterrows()] 

In [38]:
size=400
model_dm = gensim.models.Doc2Vec(min_count=1,window=10,size=size,sample=1e-3,negative=3,workers=8)

In [39]:
model_dm.build_vocab(x_train)

In [ ]:
'''
Running this will take a lot of time
'''
for epoch in range(10):
    perm = np.random.permutation(np.array(x_train).shape[0])
    model_dm.train(x_train[perm])

In [40]:
model_dm.train(x_train)

13093113

In [41]:
model_file_path = os.path.join("models","conversation_models.pkl")

In [42]:
with open(model_file_path,"wb") as f:
    pickle.dump(model_dm,f,pickle.HIGHEST_PROTOCOL)

With a model now in place, we can use it to find similar characters. The below is the code for achieving it.

In [43]:
conversations.iloc[10]

character_id_1                                                u1094
character_id_2                                                u1100
line_ids                                     ['L236546', 'L236547']
movie_id                                                        m72
dialogues_text    [yeah , right , no , i'm, serious , give, us, ...
Name: 10005, dtype: object

In [44]:
tokens =conversations.dialogues_text[10]
new_vector = model_dm.infer_vector(tokens)
sims = model_dm.docvecs.most_similar(positive=[new_vector],topn=3)

The result is that we have the top 3 similar characters and thei cosine similarity scores

In [45]:
sims

[('m159_u2466_u2476_22608', 0.5580568313598633),
 ('m437_u6572_u6574_60425', 0.5205708742141724),
 ('m196_u2997_u2999_27677', 0.5167331695556641)]

<h3>Finding some interesing dialogues !! </h3>

In [54]:
#tokens = ["i","know","what","you","did","last","summer"]
tokens = ["jurassic","lost","world"]
new_vector = model_dm.infer_vector(tokens)
sims = model_dm.docvecs.most_similar(positive=[new_vector],topn=3)

In [55]:
sims

[('m411_u6165_u6168_56996', 0.607579231262207),
 ('m198_u3031_u3034_28029', 0.6055326461791992),
 ('m171_u2621_u2632_24415', 0.6051242351531982)]

In [50]:
movies = pd.read_json("dataset"+os.sep+"movie_titles_metadata.json")

In [51]:
movies.head(3)

,genres,imdb_rating,imdb_votes,movie_id,movie_title,movie_year
0,"['comedy', 'romance']",6.9,62847,m0,10 things i hate about you,1999
1,"['adventure', 'biography', 'drama', 'history']",6.2,10421,m1,1492: conquest of paradise,1992
10,"['drama', 'mystery', 'thriller']",6.9,7252,m10,affliction,1997


In [56]:
movies[(movies.movie_id=="m411") | (movies.movie_id=="m198")|(movies.movie_id=="m171")]

,genres,imdb_rating,imdb_votes,movie_id,movie_title,movie_year
171,"['action', 'drama', 'romance', 'western']",6.9,115,m171,roughshod,1949
198,"['action', 'sci-fi', 'thriller']",6.4,28682,m198,star trek: nemesis,2002
411,"['action', 'adventure', 'family', 'sci-fi', 'a...",7.9,153737,m411,jurassic park,1993


In [92]:
tokens = ["surely","you","be","serious"]
new_vector = model_dm.infer_vector(tokens)
sims = model_dm.docvecs.most_similar(positive=[new_vector],topn=3)
sims

[('m141_u2196_u2200_20094', 0.6500850915908813),
 ('m52_u820_u822_7168', 0.6422507762908936),
 ('m35_u556_u567_4794', 0.6298543214797974)]

In [93]:
conversations[conversations.new_index=="m141_u2196_u2200_20094"]["dialogues_text"].values[0]

['hello ',
 'hello ',
 'i',
 'think',
 'you',
 'can',
 'understand',
 'why',
 'i',
 'have',
 'some',
 'serious',
 'questions',
 'about',
 'your',
 'methods ',
 'i',
 'mean ',
 'obviously',
 'it',
 'becomes',
 'much',
 'more',
 'likely',
 'that',
 "i'm",
 'going',
 'to',
 'have',
 'confusion',
 'about',
 'this',
 'when',
 'your',
 'idea',
 'of',
 'treatment',
 'is',
 'to',
 'go',
 'walking',
 'in',
 'the',
 'woods',
 'and',
 'up',
 'to',
 'make ',
 'outs ville',
 'and',
 'do',
 'all',
 'these',
 'highly',
 'romantic',
 'activities',
 '  ']

In [94]:
conversations[conversations.new_index=="m52_u820_u822_7168"]["dialogues_text"].values[0]

['why',
 ' ',
 'er',
 ' ',
 'mrs ',
 'teasdale',
 ' ',
 'this',
 'is',
 'an',
 'outrage ',
 '',
 'this',
 'man',
 'is',
 'impossible   ',
 'my',
 'course',
 'is',
 'clear   ',
 '',
 'this',
 'means',
 'war   ',
 '',
 'you',
 'runt ',
 'i',
 'still',
 'like',
 'upstart',
 'the',
 'best ']

In [95]:
conversations[conversations.new_index=="m35_u556_u567_4794"]["dialogues_text"].values[0]

['good',
 'god   you',
 "don't",
 'think',
 'there',
 'really',
 'is',
 'a',
 'bomb',
 'shelter ',
 'do',
 'you ',
 'fallout',
 'shelter ']

The performance of this model can be improved by further rounds of training and in different word permutations!